```{=latex}
\usepackage{hyperref}
\usepackage{graphicx}
\usepackage{listings}
\usepackage{textcomp}
\usepackage{fancyvrb}

\newcommand{\passthrough}[1]{\lstset{mathescape=false}#1\lstset{mathescape=true}}
\newcommand{\tightlist}{}
```

```{=latex}
\title{Best Practices for CI in Python}
\author{Moshe Zadka -- https://cobordism.com}
\date{}

\begin{document}
\begin{titlepage}
\maketitle
\end{titlepage}

\frame{\titlepage}
```

```{=latex}
\begin{frame}
\frametitle{Acknowledgement of Country}

Belmont (in San Francisco Bay Area Peninsula)

Ancestral homeland of the Ramaytush Ohlone people

\end{frame}
```

## What is CI?

```{=latex}
\begin{frame}
\frametitle{What is Continuous Integration made of?}

The gears in the machine

\end{frame}
```

```{=latex}
\begin{frame}
\frametitle{(Usual) Requirements for Continuous Integration}

Local "build":\pause

Lint\pause

Test\pause

Package

\end{frame}
```

```{=latex}
\begin{frame}
\frametitle{Continuous Integration: Run Build on Server}

When? \pause

Classic: Nightly \pause

Modern: Merge to main \pause

Advanced: Suggested patch (Pull Request, Merge Request) \pause

Exotica (Merge trains and more)

\end{frame}
```

```{=latex}
\begin{frame}
\frametitle{Continuous Integration: Patch Builds}

Focus of talk\pause

Automated gate: "is patch good?"

\end{frame}
```

```{=latex}
\begin{frame}
\frametitle{Continuous Integration: Runners}

Architecture:

\begin{itemize}
\item CI coordinator
\item CI runners
\end{itemize}

\end{frame}
```

```{=latex}
\begin{frame}
\frametitle{Continuous Integration: Build logs}

Modern frameworks:\pause

Collect live from runners\pause

Retain "forever"

\end{frame}
```

## What is Good CI?

```{=latex}
\begin{frame}
\frametitle{What makes CI good?}

Paint the target

\end{frame}
```

```{=latex}
\begin{frame}
\frametitle{CI criteria: accuracy}

Is the answer correct?

\end{frame}
```

```{=latex}
\begin{frame}
\frametitle{CI criteria: actionability}

If patch is not good, how clear is it how to fix? \pause

How to reproduce locally?

\end{frame}
```

```{=latex}
\begin{frame}
\frametitle{CI criteria: promptness}

How long does it take to answer?

\end{frame}
```

```{=latex}
\begin{frame}
\frametitle{CI criteria: cost}

Mostly the runner compute cost

\end{frame}
```

## Improving accuracy

```{=latex}
\begin{frame}
\frametitle{CI accuracy: use containers}

Container images with: \pause

Version of Python\pause

Other non-Python dependencies \pause

Pin the image tag!

\end{frame}
```

```{=latex}
\begin{frame}
\frametitle{CI accuracy: pin versions}

Test against pinned dependencies\pause

Upgrade pins in a dedicated patch\pause

(There are services)

\end{frame}
```

```{=latex}
\begin{frame}
\frametitle{CI accuracy: test quality}

Monitor and improve test quality\pause

(This is a whole 'nother talk)

\end{frame}
```

## Improving actionability

```{=latex}
\begin{frame}
\frametitle{CI actionability: set verbosity to 11}

Use verbosity options in test runners/linters/etc.\pause

Logs can be filtered more easily than unfiltered

\end{frame}
```

```{=latex}
\begin{frame}
\frametitle{CI actionability: test failure verbosity}

Test assertion failure in test for verbosity\pause

When raising exception, add details!


\end{frame}
```

```{=latex}
\begin{frame}
\frametitle{CI actionability: environmental details}

Spew details on environment to logs\pause

Environment variables\pause

Platform\pause

Etc.\pause

(Don't put secrets in environment)


\end{frame}
```

## Improving promptness

```{=latex}
\begin{frame}
\frametitle{CI promptness: caching}

Use CI primitives to cache downloads\pause

Container layer caching\pause (complicated but worth it)\pause

Local PyPI and Container caching proxies

\end{frame}
```

```{=latex}
\begin{frame}
\frametitle{CI promptness: pooling}

Reuse connections, downloads, etc.\pause

Think carefully how to break up tests

\end{frame}
```

```{=latex}
\begin{frame}
\frametitle{CI promptness: fail fast}

Order tests based on likelihood to fail\pause

Open source solutions exist

\end{frame}
```

```{=latex}
\begin{frame}
\frametitle{CI promptness: parallelize}

Use CI primitives to parallelize independent runs

\end{frame}
```

## Improving cost

```{=latex}
\begin{frame}
\frametitle{CI cost: kill useless runs}

Example: Commit added to patch

\end{frame}
```

```{=latex}
\begin{frame}
\frametitle{CI cost: stop runs early}

Do you need to finish if it fails? \pause

(Sometimes! Trade-offs)

\end{frame}
```

```{=latex}
\begin{frame}
\frametitle{CI cost: better tests}

Examples: Better stubbing/mocking instead of real services
\end{frame}
```

## Summary

```{=latex}
\begin{frame}
\frametitle{CI quality: trade-offs}

Effort \pause

Quality \pause

Cost \pause

Decide!


\end{frame}
```

```{=latex}
\begin{frame}
\frametitle{CI quality: measure}

Given trade-offs, is this ok?


\end{frame}
```

```{=latex}
\begin{frame}
\frametitle{CI quality: improve}

What needs to be better?

\end{frame}
```

```{=latex}
\begin{frame}
\frametitle{CI quality: repeat}

Constant vigilance!

\end{frame}
```

```{=latex}
\end{document}
```